In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime
%matplotlib inline
plt.rcParams['font.family'] = ['SimHei'] #  set font for chinese in matplotlib figure

In [2]:
df_policy_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/policy_0702.csv')
df_claim_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/claim_0702.csv')
df_training_raw = pd.read_csv('../Tbrain_Insurance/training-set.csv')
df_testing_raw = pd.read_csv('../Tbrain_Insurance/testing-set.csv')

In [3]:
df_policy = df_policy_raw 
df_claim = df_claim_raw 
df_training = df_training_raw
df_testing = df_testing_raw
pd.set_option('display.max_columns', None)

# Claim File
shape  (1747942, 41)

In [4]:
print(df_claim.shape) # 20 items  (69613, 20)
print('unique:',df_claim['Policy_Number'].nunique()) # 35,895/ 351,273 (10.22% 求償) unique Policy Number
print('avg num:', df_claim.shape[0]/ df_claim['Policy_Number'].nunique()) #平均 1.9 requests
print('% claim:',df_claim['Policy_Number'].nunique()/df_policy['Policy_Number'].nunique()) 

(69613, 20)
unique: 35895
avg num: 1.939350884524307
% claim: 0.10218547966965864


### 各個欄位處理方式

In [5]:
# Policy_Number	保單號碼 
# Insured's_ID	被保險人代號(替代值) 
# Prior_Policy_Number	前保單號     看 Cancellation & drop 掉
# Cancellation	非繼承自前保單  補成 0 & 1
# Vehicle_identifier	車牌號碼               # 算完ＩＤ擁有車輛數目後drop掉
# Vehicle_Make_and_Model1	廠牌名稱一  # drop 
# Vehicle_Make_and_Model2	廠牌名稱二  # drop
# Manafactured_Year_and_Month	製造年份  # 轉車齡
# Engine_Displacement_(Cubic_Centimeter)	排氣量 # drop 看重置成本
# Imported_or_Domestic_Car	車系代號 # dummy
# Coding_of_Vehicle_Branding_&_Type	廠牌車型代號 #drop
# qpt	乘載數量 # 保留值
# fpt	乘載單位  #drop
# Main_Insurance_Coverage_Group	險種分類  #dummy
# Insurance_Coverage	險種代碼 # dummy
# Insured_Amount1	保額一  # keep
# Insured_Amount2	保額二  # keep
# Insured_Amount3	保額三 # keep
# Coverage_Deductible_if_applied	自負額  # 展開成 (險種abc) x (比例, 數字) + 是否有負號, 注意percentage數值 注意是否與自付額無關的險種
# Premium	簽單保費 # 分三險種加總 + total 4欄,
# Replacement_cost_of_insured_vehicle	重置成本
# Distribution_Channel	主通路代號  # 用Distribution_Channel跑單率替代
# Multiple_Products_with_TmNewa_(Yes_or_No?)	同一車主前一年度非車險保單件數 # keep
# lia_class	車責本期級數  # keep
# plia_acc	車責賠款係數  # +1 keep
# pdmg_acc	車體賠款係數  # +1 keep
# fassured	被保險人性質 # dummy
# ibirth	出生日期  # 轉年齡 自然人ＫＮＮ補 自然人drop
# fsex	性別  # 法人model drop
# fmarriage	婚姻狀況 # 同上
# aassured_zip	郵遞區號 # drop
# iply_area	承保地區代號(分公司)  #dummy
# dbirth	要保人出生日期  # 同出生日期
# fequipment1	配備-音響註記  #dummy
# fequipment2	配備-車箱註記  #dummy
# fequipment3	配備-冷凍機組註記  #dummy
# fequipment4	配備-車框、車斗註記  # drop
# fequipment5	配備-昇降系統註記  #dummy
# fequipment6	配備-電動車電池註記 # drop
# fequipment9	配備-其他  #dummy
# nequipment9	配備-其他說明 #dummy blank->.0  not blank -> 1
##################################### 生feature
# Insured's_ID有幾輛車
# Insured's_ID有張保單
# Insured's_ID 車責 竊盜 車損 單數的 sum, mean, std
# Main_Insurance_Coverage_Group count
# Insurance_Coverage count
# sum, std, mean
# Insured_Amount1_車責 Insured_Amount1_竊盜 Insured_Amount1_車損
# Insured_Amount2 同上
# Insured_Amount3 同上
# (Insured_Amount1 + Insured_Amount2 + Insured_Amount3) * 3 (Insurance_Coverage)
# 3險種  total 保額  / total 保費
# total保費 / 重置成本
# Distribution_Channel 跑單率
# lia_class 係數轉換表 (x-4) * 0.1 + 1
# fequipment1~fequipment9, nequipment9 sum != 0
# ibirth != dbirth

In [6]:
df_claim.sample(10)

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time
36728,6004b8c619dfa915a5338efda45db69c210c0387,1,bdea9f37e5610fb584d607292a25bb582814d2c4,1,3,11/1964,1,2016/05,17ba0791499db908433b80f37c5fbc89b870084b,6740,0,0,16P,b51edab483693117cb70b8d45615fce519cda3e3,100.0,1,0,bd307a3ec329e10a2cff8fb87480823da114f8f4,1.981,10:00
3163,9c4e56e1261352b6d388b4361bea469aa40005d7,1,4b52f7343ead0898067067f7abd3b8fa78688093,2,1,02/1984,2,2015/08,17ba0791499db908433b80f37c5fbc89b870084b,18960,700,0,16G,04153228f578a349cbb8c8be3d341d75aee9eb0b,100.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,11:00
65242,f4db94babd7f8fa7305fbcaab5c70d93ab6be9a3,1,b6acf506a26db103fe3d5f91e4effc0e57dbeb3c,2,2,01/1992,2,2016/01,17ba0791499db908433b80f37c5fbc89b870084b,14280,0,0,04M,2fd007820ebca1e75f101b7bbeeadd0ae67c6103,30.0,1,0,c66c65175fecc3103b3b587be9b5b230889c8628,1.981,17:00
1551,f767449505a5cca5642c0dfd97be29cb67b3d82f,1,a344e2ea7dec619d8f26bd8585945fd1bc67026f,2,5,08/1970,1,2015/05,17ba0791499db908433b80f37c5fbc89b870084b,10039,900,0,16P,e358dea4a957dabc387be23fa1f938eb33d0c004,100.0,1,0,bd307a3ec329e10a2cff8fb87480823da114f8f4,1.981,18:00
33118,a7ff18a6cdc6f87f5e9a7533364634faf9fdbbc8,1,965ff001c938e6a6cc36423028a11ee8164d97cc,1,2,05/1986,1,2015/12,17ba0791499db908433b80f37c5fbc89b870084b,30000,832,0,16P,0ea1140d85b381f470fbea08c3540459ce605846,70.0,1,0,bd307a3ec329e10a2cff8fb87480823da114f8f4,1.981,05:00
5879,ff0aab170e9f521bca5222eafc7b097415a45c28,1,8f71c8bbd52ca97bef23025dda7906e3ce7913fa,1,2,03/1974,1,2015/09,17ba0791499db908433b80f37c5fbc89b870084b,25030,0,0,55J,7c6d632930affd3e0db2d4bec85117b3e371fe0d,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.787,07:00
17793,30b365777911eb40d728ee865f4272a4c9ebba4c,1,acde08adb56e946637e82f07edaa04456dba1c06,2,1,09/1977,1,2016/07,17ba0791499db908433b80f37c5fbc89b870084b,5050,700,0,16P,dede7b155a66a5e9d4bd0567ba9dd24fda09c19c,100.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,08:00
21575,7267b40f4440f58de24c5a981418147ddcc9176f,1,76096f5fb10b9a7ee45ad026c96f2864891ebc73,1,1,07/1956,2,2015/08,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,4c051506bade94fdd67c3f32f4a0cf95877f0c72,0.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.551,08:00
7920,df3f23489ef10c73ff940d7f2a8504cef68fc30b,1,d242a30c6b207f7ac5f6fa9030259e67d943dbce,1,1,12/1970,2,2015/12,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,940e901e3dfb0c2c833e7e51955a48b0d6d8a2f5,70.0,1,0,c1dfd96eea8cc2b62785275bca38ac261256e278,2.146,19:00
41293,fe54684efcef8b3cbb75e67fe7530ef62ac5a204,1,f161318beb02d705f9f276ad95207522ce204452,1,2,03/1960,1,2015/07,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,36448f033777e0c0a9e16323314f24cea5a63298,70.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.981,11:00


In [7]:
A = ['12L','14E','15F','15O','16G','16P','18@','18I','25G','26H','27I','29B','29K','37J','40M','41E','41N','42F','46A','47B','57L','65K','70G','70P','71H','72@']
B = ['00I','01A','01J','02K','03L','04M','05E','06F','07P','08H','14N','20B','20K','32N','33F','33O','34P','35H','36I','45@','51O','55J','56B','56K','57C','66C','66L','67D']
C = ['05N','09@','09I','10A','68E','68N']
print(len(A))
print(len(B))
print(len(C))

26
28
6


In [8]:
# Claim_Number 賠案號碼          count() nunique() per Policy_number
# Nature_of_the_claim 賠案性質    NoC1 Noc2 count() nunique() per Policy_number
# Policy_Number 保單號碼                count
# Driver's_Gender 肇事駕駛性別                 dummy 
# Driver's_Relationship_with_Insured 與被保險人關係    dummy
# DOB_of_Driver 肇事駕駛出生日                             age
# Marital_Status_of_Driver 肇事駕駛婚姻             dummy
# Accident_Date 出險日期                          Day Month weekday 
# Cause_of_Loss 出險原因                                  dummy
# Paid_Loss_Amount 已決賠款               (1)NoC1/2 sum  sumABC
# paid_Expenses_Amount 已決費用            (2)NoC1/2 sum 
# Salvage_or_Subrogation? 追償金額          (3)NoC1/2 sum 
#         4 = 1+2+3
#         4 = 1+2-3
#       switch NoC1+ NoC2-
#    / InsuredGroup InsuredO PremiumGroup PremiumO

# 1st + Main_insured_group  ->  Coverage 險種代號      dummy                    
# Vehicle_identifier 車牌        count map V_I
# At_Fault? 肇責比例                   keep  sum?mean? ABC
# ClaimStatus(close,_open,_reopen_etc) 單一險種CLOSE  dummy
# Deductible 自負額
# Accident_area 出險地區                   drop
# number_of_claimants 事故關係人人數      keep 
# Accident_Time 出險時間                H    abs(-mean )

In [9]:
#    / InsuredGroup InsuredO PremiumGroup PremiumO   **********

In [10]:
df_claim ['group'] = df_claim.apply(lambda row : 'A' if row['Coverage'] in A 
                                    else ('C' if row['Coverage'] in C else 'B'), axis = 1)
df_claim.head()

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time,group
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,B
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00,A
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00,A


#### claim_number

In [11]:
df_claim_grouped = df_claim.groupby(['Policy_Number'])[['Claim_Number']].nunique().reset_index()
df_claim = df_claim.merge(df_claim_grouped, on = ['Policy_Number'], how = 'left', suffixes=('','_unuique'))
df_claim.head()

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time,group,Claim_Number_unuique
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,B,2
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00,A,1
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00,A,1


In [12]:
df_claim_grouped = df_claim.groupby(['Policy_Number'])[['Claim_Number']].count().reset_index()
df_claim = df_claim.merge(df_claim_grouped, on = ['Policy_Number'], how = 'left', suffixes=('','_count'))
df_claim.head()

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time,group,Claim_Number_unuique,Claim_Number_count
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,B,2,5
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2,5
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2,5
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00,A,1,1
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00,A,1,2


In [13]:
print(df_claim.shape)
df_claim_grouped = df_claim.groupby(['Policy_Number','Nature_of_the_claim'])[['Claim_Number']].count().reset_index()
df_claim = df_claim.merge(df_claim_grouped, on = ['Policy_Number','Nature_of_the_claim'], how = 'left', suffixes=('','_typecount'))
print(df_claim.shape)
df_claim.head()

(69613, 23)
(69613, 24)


,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time,group,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,B,2,5,5
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2,5,5
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2,5,5
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00,A,1,1,1
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00,A,1,2,2


In [14]:
print(df_claim.shape)
df_claim_grouped = df_claim.groupby(['Policy_Number','Nature_of_the_claim'])[['Claim_Number']].nunique().reset_index()
df_claim = df_claim.merge(df_claim_grouped, on = ['Policy_Number','Nature_of_the_claim'], how = 'left', suffixes=('','_typenunique'))
print(df_claim.shape)
df_claim.head()

(69613, 24)
(69613, 25)


,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time,group,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,B,2,5,5,2
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2,5,5,2
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,A,2,5,5,2
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00,A,1,1,1,1
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00,A,1,2,2,1


In [15]:
segment = ['Nature_of_the_claim', 'group']
print('Shape of data before get dummy : {}'.format(df_claim.shape))
df_claim_dummy = pd.get_dummies(df_claim, columns = segment)
print('Shape of data after get dummy : {}'.format(df_claim_dummy.shape))

Shape of data before get dummy : (69613, 25)
Shape of data after get dummy : (69613, 28)


In [16]:
df_claim = df_claim_dummy
df_claim.head()

,Claim_Number,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,2,5,5,2,1,0,0,1,0
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,2,5,5,2,1,0,1,0,0
2,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,2,5,5,2,1,0,1,0,0
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00,1,1,1,1,1,0,1,0,0
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00,1,2,2,1,1,0,1,0,0


In [17]:
# count
df_claim['Claim_Number_type1count'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_1']
df_claim['Claim_Number_type2count'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_2']

df_claim['Claim_Number_typeAcount'] = df_claim['Claim_Number_typecount']*df_claim['group_A']
df_claim['Claim_Number_typeBcount'] = df_claim['Claim_Number_typecount']*df_claim['group_B']
df_claim['Claim_Number_typeCcount'] = df_claim['Claim_Number_typecount']*df_claim['group_C']

df_claim['Claim_Number_type1Acount'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_1']*df_claim['group_A']
df_claim['Claim_Number_type1Bcount'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_1']*df_claim['group_B']
df_claim['Claim_Number_type1Ccount'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_1']*df_claim['group_C']

df_claim['Claim_Number_type2Acount'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_2']*df_claim['group_A']
df_claim['Claim_Number_type2Bcount'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_2']*df_claim['group_B']
df_claim['Claim_Number_type2Ccount'] = df_claim['Claim_Number_typecount']*df_claim['Nature_of_the_claim_2']*df_claim['group_C']

# Uni Count 
df_claim['Claim_Number_type1UniCount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_1']
df_claim['Claim_Number_type2UniCount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_2']

df_claim['Claim_Number_typeAUniCount'] = df_claim['Claim_Number_typenunique']*df_claim['group_A']
df_claim['Claim_Number_typeBUniCount'] = df_claim['Claim_Number_typenunique']*df_claim['group_B']
df_claim['Claim_Number_typeCUniCount'] = df_claim['Claim_Number_typenunique']*df_claim['group_C']

df_claim['Claim_Number_type1UniCcount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_1']*df_claim['group_A']
df_claim['Claim_Number_type1UniCcount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_1']*df_claim['group_B']
df_claim['Claim_Number_type1UniCcount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_1']*df_claim['group_C']

df_claim['Claim_Number_type2UniCcount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_2']*df_claim['group_A']
df_claim['Claim_Number_type2UniCcount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_2']*df_claim['group_B']
df_claim['Claim_Number_type2UniCcount'] = df_claim['Claim_Number_typenunique']*df_claim['Nature_of_the_claim_2']*df_claim['group_C']

df_claim.shape

(69613, 46)

### get dummy on : 

In [18]:
dummy_cols = ["Driver's_Gender", "Driver's_Relationship_with_Insured", "Marital_Status_of_Driver", 
              "Cause_of_Loss", "Claim_Status_(close,_open,_reopen_etc)",'Coverage']

print('Shape of data before get dummy : {}'.format(df_claim.shape))

df_claim_dummy = pd.get_dummies(df_claim, columns = dummy_cols)
print('Shape of data after get dummy : {}'.format(df_claim_dummy.shape))

Shape of data before get dummy : (69613, 46)
Shape of data after get dummy : (69613, 118)


In [19]:
df_claim = df_claim_dummy

### Paid

In [20]:
# Paid_Loss_Amount 已決賠款               (1)NoC1/2 sum  sumABC
# paid_Expenses_Amount 已決費用            (2)NoC1/2 sum 
# Salvage_or_Subrogation? 追償金額          (3)NoC1/2 sum 
#         4 = 1+2+3
#         4 = 1+2-3
#       switch NoC1+ NoC2-

In [21]:
df_claim[['Claim_Number','Policy_Number','Paid_Loss_Amount','paid_Expenses_Amount',"Salvage_or_Subrogation?",'group_A','group_B','group_C']].head()

,Claim_Number,Policy_Number,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,group_A,group_B,group_C
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,56705,0,0,0,1,0
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,30000,700,0,1,0,0
2,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1100,0,0,1,0,0
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,151795,700,0,1,0,0
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,0,0,0,1,0,0


In [22]:
## 1.Paid loss
df_claim['PL_1A'] = df_claim['Paid_Loss_Amount']*df_claim['Nature_of_the_claim_1']*df_claim['group_A']
df_claim['PL_2A'] = df_claim['Paid_Loss_Amount']*df_claim['Nature_of_the_claim_2']*df_claim['group_A']
df_claim['PL_1B'] = df_claim['Paid_Loss_Amount']*df_claim['Nature_of_the_claim_1']*df_claim['group_B']
df_claim['PL_2B'] = df_claim['Paid_Loss_Amount']*df_claim['Nature_of_the_claim_2']*df_claim['group_B']
df_claim['PL_1C'] = df_claim['Paid_Loss_Amount']*df_claim['Nature_of_the_claim_1']*df_claim['group_C']
df_claim['PL_2C'] = df_claim['Paid_Loss_Amount']*df_claim['Nature_of_the_claim_2']*df_claim['group_C']

## 2.Paid Exp
df_claim['PE_1A'] = df_claim['paid_Expenses_Amount']*df_claim['Nature_of_the_claim_1']*df_claim['group_A']
df_claim['PE_2A'] = df_claim['paid_Expenses_Amount']*df_claim['Nature_of_the_claim_2']*df_claim['group_A']
df_claim['PE_1B'] = df_claim['paid_Expenses_Amount']*df_claim['Nature_of_the_claim_1']*df_claim['group_B']
df_claim['PE_2B'] = df_claim['paid_Expenses_Amount']*df_claim['Nature_of_the_claim_2']*df_claim['group_B']
df_claim['PE_1C'] = df_claim['paid_Expenses_Amount']*df_claim['Nature_of_the_claim_1']*df_claim['group_C']
df_claim['PE_2C'] = df_claim['paid_Expenses_Amount']*df_claim['Nature_of_the_claim_2']*df_claim['group_C']

## 3.Subrogation
df_claim['Sub_1A'] = df_claim["Salvage_or_Subrogation?"]*df_claim['Nature_of_the_claim_1']*df_claim['group_A']
df_claim['Sub_2A'] = df_claim["Salvage_or_Subrogation?"]*df_claim['Nature_of_the_claim_2']*df_claim['group_A']
df_claim['Sub_1B'] = df_claim["Salvage_or_Subrogation?"]*df_claim['Nature_of_the_claim_1']*df_claim['group_B']
df_claim['Sub_2B'] = df_claim["Salvage_or_Subrogation?"]*df_claim['Nature_of_the_claim_2']*df_claim['group_B']
df_claim['Sub_1C'] = df_claim["Salvage_or_Subrogation?"]*df_claim['Nature_of_the_claim_1']*df_claim['group_C']
df_claim['Sub_2C'] = df_claim["Salvage_or_Subrogation?"]*df_claim['Nature_of_the_claim_2']*df_claim['group_C']


## overall 
#  1' = 1 + 2 + 3
df_claim['1A_Sum_1'] = df_claim['PL_1A'] + df_claim['PE_1A'] + df_claim['Sub_1A']
df_claim['2A_Sum_1'] = df_claim['PL_2A'] + df_claim['PE_2A'] + df_claim['Sub_2A']
df_claim['1B_Sum_1'] = df_claim['PL_1B'] + df_claim['PE_1B'] + df_claim['Sub_1B']
df_claim['2B_Sum_1'] = df_claim['PL_2B'] + df_claim['PE_2B'] + df_claim['Sub_2B']
df_claim['1C_Sum_1'] = df_claim['PL_1C'] + df_claim['PE_1C'] + df_claim['Sub_1C']
df_claim['2C_Sum_1'] = df_claim['PL_2C'] + df_claim['PE_2C'] + df_claim['Sub_2C']

#  2' = 1 + 2 - 3
df_claim['1A_Sum_2'] = df_claim['PL_1A'] + df_claim['PE_1A'] - df_claim['Sub_1A']
df_claim['2A_Sum_2'] = df_claim['PL_2A'] + df_claim['PE_2A'] - df_claim['Sub_2A']
df_claim['1B_Sum_2'] = df_claim['PL_1B'] + df_claim['PE_1B'] - df_claim['Sub_1B']
df_claim['2B_Sum_2'] = df_claim['PL_2B'] + df_claim['PE_2B'] - df_claim['Sub_2B']
df_claim['1C_Sum_2'] = df_claim['PL_1C'] + df_claim['PE_1C'] - df_claim['Sub_1C']
df_claim['2C_Sum_2'] = df_claim['PL_2C'] + df_claim['PE_2C'] - df_claim['Sub_2C']

#  3' = 1 + 2 - 3 (2)/ 1 + 2 + 3 (1)
df_claim['1A_Sum_3'] = df_claim['PL_1A'] + df_claim['PE_1A'] + df_claim['Sub_1A']
df_claim['2A_Sum_3'] = df_claim['PL_2A'] + df_claim['PE_2A'] - df_claim['Sub_2A']
df_claim['1B_Sum_3'] = df_claim['PL_1B'] + df_claim['PE_1B'] + df_claim['Sub_1B']
df_claim['2B_Sum_3'] = df_claim['PL_2B'] + df_claim['PE_2B'] - df_claim['Sub_2B']
df_claim['1C_Sum_3'] = df_claim['PL_1C'] + df_claim['PE_1C'] + df_claim['Sub_1C']
df_claim['2C_Sum_3'] = df_claim['PL_2C'] + df_claim['PE_2C'] - df_claim['Sub_2C']

print(df_claim.shape)

(69613, 154)


In [23]:
cols = ['PL_1A','PL_2A','PL_1B','PL_2B','PL_1C','PL_2C',
        'PE_1A','PE_2A','PE_1B','PE_2B','PE_1C','PE_2C',
        'Sub_1A','Sub_2A','Sub_1B','Sub_2B','Sub_1C','Sub_2C',
       '1A_Sum_1','2A_Sum_1','1B_Sum_1','2B_Sum_1','1C_Sum_1','2C_Sum_1',
       '1A_Sum_2','2A_Sum_2','1B_Sum_2','2B_Sum_2','1C_Sum_2','2C_Sum_2',
       '1A_Sum_3','2A_Sum_3','1B_Sum_3','2B_Sum_3','1C_Sum_3','2C_Sum_3']

df_claim_grouped = df_claim.groupby(['Policy_Number','Claim_Number'])[cols].sum().reset_index()
df_claim_grouped.head()                                                     

,Policy_Number,Claim_Number,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3
0,0004a3f30deef9fc8f9f5cb334b8bec8d79c7453,dfc5b41594b0756e98651c0f250d5fc1d2035141,0,0,79600,0,0,0,0,0,3000,950,0,0,0,0,0,42800,0,0,0,0,82600,43750,0,0,0,0,82600,-41850,0,0,0,0,82600,-41850,0,0
1,0009049ed3ac539074a3cbfadb59cac6aec2fd98,d3b45bd38b3d42b6f39ff858990f71094ffd082f,0,0,40315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40315,0,0,0,0,0,40315,0,0,0,0,0,40315,0,0,0
2,000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad,8c227e996c2d28f803f113a1c79c14d523abc490,0,0,18286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18286,0,0,0,0,0,18286,0,0,0,0,0,18286,0,0,0
3,000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad,f300d77f5197058f387182e9c57bac606a9ba76d,0,0,10555,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10555,0,0,0,0,0,10555,0,0,0,0,0,10555,0,0,0
4,0009dfd25805f44c7fa82c1ddb010eb3a05612f6,662cf5de0bce63c63dbba4b780e43aef26d38aa4,13496,0,0,0,0,0,0,0,700,0,0,0,0,0,0,0,0,0,13496,0,700,0,0,0,13496,0,700,0,0,0,13496,0,700,0,0,0


In [24]:
## merge back the sum() to policy 
df_claim_temp = df_claim.merge(df_claim_grouped, on = ['Policy_Number','Claim_Number'], how = 'left', suffixes=('','_sum'))
print(df_claim_temp.shape)
df_claim_temp.head(10)

(69613, 190)


,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,Accident_area,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,PL_1A_sum,PL_2A_sum,PL_1B_sum,PL_2B_sum,PL_1C_sum,PL_2C_sum,PE_1A_sum,PE_2A_sum,PE_1B_sum,PE_2B_sum,PE_1C_sum,PE_2C_sum,Sub_1A_sum,Sub_2A_sum,Sub_1B_sum,Sub_2B_sum,Sub_1C_sum,Sub_2C_sum,1A_Sum_1_sum,2A_Sum_1_sum,1B_Sum_1_sum,2B_Sum_1_sum,1C_Sum_1_sum,2C_Sum_1_sum,1A_Sum_2_sum,2A_Sum_2_sum,1B_Sum_2_sum,2B_Sum_2_sum,1C_Sum_2_sum,2C_Sum_2_sum,1A_Sum_3_sum,2A_Sum_3_sum,1B_Sum_3_sum,2B_Sum_3_sum,1C_Sum_3_sum,2C_Sum_3_sum
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,06/1969,2015/01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56

### drop accident area

In [25]:
print(df_claim.shape)
df_claim_drop = df_claim.drop(['Accident_area'], axis = 1)
print(df_claim_drop.shape)

(69613, 154)
(69613, 153)


In [26]:
df_claim = df_claim_drop

### datetime 

In [27]:
##### DOB_of_Driver 肇事駕駛出生日                             age
##### Accident_Date 出險日期                          Day Month weekday
##### Accident_Time 出險時間                H    abs(-mean )

In [28]:
date_cols = ['DOB_of_Driver','Accident_Date','Accident_Time']
print(df_claim[date_cols].isnull().sum())
df_claim[date_cols].head()

DOB_of_Driver    0
Accident_Date    0
Accident_Time    0
dtype: int64


,DOB_of_Driver,Accident_Date,Accident_Time
0,06/1969,2015/01,08:00
1,06/1969,2015/01,08:00
2,06/1969,2015/01,08:00
3,09/1975,2015/01,08:00
4,02/1971,2015/01,08:00


In [29]:
## convert to datetime formate
df = df_claim

date_col=['DOB_of_Driver']
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%m/%Y', errors='ignore')  #'%Y-%m-%d %H:%M:%S'   #errors='coerce'

date_col=['Accident_Date']
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%Y/%m', errors='ignore')  #'%Y-%m-%d %H:%M:%S'   #errors='coerce'

date_col=['Accident_Time']
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%H:%M', errors='ignore')  #'%Y-%m-%d %H:%M:%S'   #errors='coerce'

df[date_cols].info(verbose= True)
df_claim = df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69613 entries, 0 to 69612
Data columns (total 3 columns):
DOB_of_Driver    69613 non-null datetime64[ns]
Accident_Date    69613 non-null datetime64[ns]
Accident_Time    69613 non-null datetime64[ns]
dtypes: datetime64[ns](3)
memory usage: 2.1 MB


#### accident date (day, weekday, date, month)

In [30]:
df = df_claim_raw
df[['AYear','AMonth']]= df['Accident_Date'].str.split('/',expand=True).apply(lambda x: x.str.strip())
df[['AYear','AMonth']].shape

(69613, 2)

In [31]:
print(df_claim.shape)
df_claim[['AYear','AMonth']] = df[['AYear','AMonth']]
print(df_claim.shape)
df_claim.head()

(69613, 153)
(69613, 155)


,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,01
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,30000,700,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,0,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,1,0,0,5,0,5,0,0,5,0,0,0,0,0,2,0,2,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30000

In [32]:
df = df_claim
from datetime import datetime, date
df['Acc_DayOfWeek'] = df.apply(lambda row: row['Accident_Date'].weekday(), axis =1)

df_claim = df
df['Acc_DayOfWeek'][:5]  

0    3
1    3
2    3
3    3
4    3
Name: Acc_DayOfWeek, dtype: int64

In [33]:
# calculate
df = df_claim_raw
df[['AHour','Amin']]= df['Accident_Time'].str.split(':',expand=True).apply(lambda x: x.str.strip())
print(df[['AHour','Amin']].shape)

# merge: 
print(df_claim.shape)
df_claim[['AHour','Amin']] = df[['AHour','Amin']]
print(df_claim.shape)

(69613, 2)
(69613, 156)
(69613, 158)


### Driver age

In [34]:
#  https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.datetime.html
#  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value

In [35]:
# calculate years 
Cal_year = pd.to_datetime('01/2017', format ='%m/%Y')
daysdiff = Cal_year - df_claim['DOB_of_Driver']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
years = components.days/365
df_claim['Dr_age'] = years
df_claim[['DOB_of_Driver','Dr_age']].head()

,DOB_of_Driver,Dr_age
0,1969-06-01,47.619178
1,1969-06-01,47.619178
2,1969-06-01,47.619178
3,1975-09-01,41.364384
4,1971-02-01,45.947945


In [38]:
df_claim.shape

(69613, 159)

###  Vehicle_identifier 車牌        count map V_I

In [36]:
df_claim_grouped = df_claim.groupby(['Vehicle_identifier'])['Claim_Number','Policy_Number'].count().reset_index()
df_claim_grouped.head()

,Vehicle_identifier,Claim_Number,Policy_Number
0,000130954365e7ca293a5ed44f33ea15317ae588,2,2
1,000595fbde3ea724ebbf2cad8c1d8f0bcacc2f6a,3,3
2,000a76307e8769ab9e741fc642dd5add96b1635d,1,1
3,000b31d46d04cda57a802e8958b4957c11474cb0,1,1
4,000ca9b5503d02be6f7db55f804f37cd0e39c6b0,8,8
5,000dffb2d79e1f274f43a5c7f7ba846af594bd07,1,1
6,00103f70bcf7184d7ac907204d78c36c90e9389f,2,2
7,0014146c958c8fe3b9406c3ff132314163d66b20,1,1
8,0014daca7816df6715f8e00c0c068d263b603d61,2,2
9,0017f90f3d70015280d35e7f22a5932107aa434d,3,3


In [40]:
## merge back the sum() to policy 
df_claim_temp = df_claim.merge(df_claim_grouped, on = ['Vehicle_identifier'], how = 'left', suffixes=('','_onPlateNumber'))
print(df_claim_temp.shape)
df_claim_temp.head(10)

(69613, 161)


,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,01,3,08,00,47.619178,5.0,5.0
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,30000,700,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,0,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,1,0,0,5,0,5,0,0,5,0,0,0,0,0,2,0,2,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [43]:
df_claim = df_claim_temp

### At_Fault?

In [44]:
df_claim['AF_A'] = df_claim["At_Fault?"] * df_claim['group_A']
df_claim['AF_B'] = df_claim["At_Fault?"] * df_claim['group_B']
df_claim['AF_C'] = df_claim["At_Fault?"] * df_claim['group_C']
df_claim.shape

(69613, 164)

In [45]:
df_claim_grouped_sum = df_claim.groupby(['Policy_Number'])['AF_A','AF_B','AF_C'].sum().reset_index()
df_claim_grouped_sum.head()

,Policy_Number,AF_A,AF_B,AF_C
0,0004a3f30deef9fc8f9f5cb334b8bec8d79c7453,150.0,200.0,0.0
1,0009049ed3ac539074a3cbfadb59cac6aec2fd98,0.0,100.0,0.0
2,000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad,70.0,170.0,0.0
3,0009dfd25805f44c7fa82c1ddb010eb3a05612f6,200.0,100.0,0.0
4,000a5d472fa501c2e9d771dfae4af646f281f97d,200.0,200.0,0.0


In [46]:
df_claim_grouped_mean = df_claim.groupby(['Policy_Number'])['AF_A','AF_B','AF_C'].mean().reset_index()
df_claim_grouped_mean.head()

,Policy_Number,AF_A,AF_B,AF_C
0,0004a3f30deef9fc8f9f5cb334b8bec8d79c7453,21.428571,28.571429,0.0
1,0009049ed3ac539074a3cbfadb59cac6aec2fd98,0.000000,100.000000,0.0
2,000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad,23.333333,56.666667,0.0
3,0009dfd25805f44c7fa82c1ddb010eb3a05612f6,66.666667,33.333333,0.0
4,000a5d472fa501c2e9d771dfae4af646f281f97d,50.000000,50.000000,0.0


In [47]:
## merge back the sum() to policy 
df_claim_temp = df_claim.merge(df_claim_grouped_sum, on = ['Policy_Number'], how = 'left', suffixes=('','_sum'))
print(df_claim_temp.shape)
df_claim_temp.head()

(69613, 167)


,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber,AF_A,AF_B,AF_C,AF_A_sum,AF_B_sum,AF_C_sum
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,01,3,08,00,47.619178,5.0,5.0,0.0,100.0,0.0,200.0,100.0,0.0
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,30000,700,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,0,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,1,0,0,5,0,5,0,0,5,

In [48]:
## merge back the sum() to policy 
df_claim_temp = df_claim_temp.merge(df_claim_grouped_mean, on = ['Policy_Number'], how = 'left', suffixes=('','_mean'))
print(df_claim_temp.shape)
df_claim_temp.head()

(69613, 170)


,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber,AF_A,AF_B,AF_C,AF_A_sum,AF_B_sum,AF_C_sum,AF_A_mean,AF_B_mean,AF_C_mean
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,01,3,08,00,47.619178,5.0,5.0,0.0,100.0,0.0,200.0,100.0,0.0,40.0,20.0,0.0
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,30000,700,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,0,1.981,1900-0

In [62]:
df_claim = df_claim_temp
df_claim.head()

,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber,AF_A,AF_B,AF_C,AF_A_sum,AF_B_sum,AF_C_sum,AF_A_mean,AF_B_mean,AF_C_mean
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,01,3,08,00,47.619178,5.0,5.0,0.0,100.0,0.0,200.0,100.0,0.0,40.0,20.0,0.0
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,30000,700,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,0,1.981,1900-0

### Deductible

In [66]:
df_claim['DeductibleA'] = df_claim['Deductible'] * df_claim['group_A']
df_claim['DeductibleB'] = df_claim['Deductible'] * df_claim['group_B']
df_claim['DeductibleC'] = df_claim['Deductible'] * df_claim['group_C']

df_claim_grouped_sum = df_claim.groupby(['Claim_Number','Policy_Number'])['Deductible','DeductibleA','DeductibleB','DeductibleC'].sum().reset_index()
df_claim_grouped_sum.head()

,Claim_Number,Policy_Number,Deductible,DeductibleA,DeductibleB,DeductibleC
0,00003407e9e84870a310ef4f3c37a0e04925ac44,e30405a30ab6f06fe1e2809335adfb6e246fd1b6,0,0,0,0
1,00042ea566baae337f2da427bc9ee682b1560314,c43472f5c44ff6aaf7fd03d81e992dc0e2f21312,0,0,0,0
2,00082cdef63b85de838431628d0721868caba28b,9645d04fad719ffba45c9e737983c7ad1eb11cc9,0,0,0,0
3,0008d93f152a2e3b18a2d41a6e3aefad60b2815a,af1c44e65e1ecaf800b7da5b020d32d198d8c341,0,0,0,0
4,000988c4569f0180c9bedeaf2bb15f06ac37da1c,4378fdb968ea6104fc6a4b07a1964372108df9b3,0,0,0,0


In [70]:
df_claim_grouped_mean = df_claim.groupby(['Claim_Number','Policy_Number'])['Deductible','DeductibleA','DeductibleB','DeductibleC'].mean().reset_index()
df_claim_grouped_mean.head()

,Claim_Number,Policy_Number,Deductible,DeductibleA,DeductibleB,DeductibleC
0,00003407e9e84870a310ef4f3c37a0e04925ac44,e30405a30ab6f06fe1e2809335adfb6e246fd1b6,0.0,0.0,0.0,0.0
1,00042ea566baae337f2da427bc9ee682b1560314,c43472f5c44ff6aaf7fd03d81e992dc0e2f21312,0.0,0.0,0.0,0.0
2,00082cdef63b85de838431628d0721868caba28b,9645d04fad719ffba45c9e737983c7ad1eb11cc9,0.0,0.0,0.0,0.0
3,0008d93f152a2e3b18a2d41a6e3aefad60b2815a,af1c44e65e1ecaf800b7da5b020d32d198d8c341,0.0,0.0,0.0,0.0
4,000988c4569f0180c9bedeaf2bb15f06ac37da1c,4378fdb968ea6104fc6a4b07a1964372108df9b3,0.0,0.0,0.0,0.0


In [71]:
## merge back the sum() to policy 
df_claim_temp = df_claim.merge(df_claim_grouped_sum, on = ['Policy_Number'], how = 'left', suffixes=('','_sum'))
print(df_claim_temp.shape)
df_claim_temp.head()

(69613, 176)


,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber,AF_A,AF_B,AF_C,AF_A_sum,AF_B_sum,AF_C_sum,AF_A_mean,AF_B_mean,AF_C_mean,DeductibleA,DeductibleB,DeductibleC,AF_A_sum,AF_B_sum,AF_C_sum
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,01,3,08,00,47.619178,5.0,5.0,0.0,100.0,0.0,200.0,100.0,0.0,40.0,20.0,0.0,0,3000,0,200.0,100.0,0.0
1,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06

In [72]:
## merge back the sum() to policy 
df_claim_temp = df_claim_temp.merge(df_claim_grouped_mean, on = ['Policy_Number'], how = 'left', suffixes=('','_mean'))
print(df_claim_temp.shape)
df_claim_temp.head()

(91311, 181)


,Claim_Number,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Vehicle_identifier,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber,AF_A,AF_B,AF_C,AF_A_sum,AF_B_sum,AF_C_sum,AF_A_mean,AF_B_mean,AF_C_mean,DeductibleA,DeductibleB,DeductibleC,AF_A_sum,AF_B_sum,AF_C_sum,Claim_Number_mean,Deductible_mean,DeductibleA_mean,DeductibleB_mean,DeductibleC_mean
0,954ff50c11e2c6f6bf91514fdcca98578128da21,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,86ada5177903be297607713f3e0128e7b3f8a839,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,01,3,08,00,47.619178,5.0,5.0,0.0,100.0,0.0,200.0,100.0,0.0,40.0,20.0,0.0,0,3000,0,200.0,100.0,0.0,67f782

In [73]:
df_claim = df_claim_temp

In [74]:
# df_claim.to_csv('../Tbrain_Insurance/df_claim.csv', index = False)

## fillna

In [82]:
nan_count = df_claim.isnull().sum()
nan_count[nan_count > 0]

Vehicle_identifier             516
At_Fault?                       58
Claim_Number_onPlateNumber     516
Policy_Number_onPlateNumber    516
AF_A                            58
AF_B                            58
AF_C                            58
AF_A_sum                        32
AF_B_sum                        32
AF_C_sum                        32
AF_A_mean                       32
AF_B_mean                       32
AF_C_mean                       32
AF_A_sum                        32
AF_B_sum                        32
AF_C_sum                        32
dtype: int64

In [84]:
df_claim = df_claim.fillna(0)
df_claim.isnull().sum().sum()

0

# Deal with Columns

In [90]:
df_claim.info()
'''
Claim_Number                                              object  drop  
Policy_Number                                             object  ignore
Vehicle_identifier                                        object  drop
AYear                                                     object  convert to Int
AMonth                                                    object  convert to Int
AHour                                                     object  convert to Int
Amin                                                      object  convert to Int
Claim_Number_mean                                        object   drop
'''


<class 'pandas.core.frame.DataFrame'>
Int64Index: 91311 entries, 0 to 91310
Columns: 181 entries, Claim_Number to DeductibleC_mean
dtypes: datetime64[ns](3), float64(21), int64(66), object(8), uint8(83)
memory usage: 78.7+ MB


In [95]:
print(df_claim.shape)
df_claim_drop = df_claim.drop(['Claim_Number','Vehicle_identifier','Claim_Number_mean'], axis = 1)
print(df_claim_drop.shape)

(91311, 179)
(91311, 178)


In [96]:
df_claim = df_claim_drop

In [104]:
convert_cols= ['AYear','AMonth','AHour','Amin']
for col in convert_cols:
    df_claim[col] = pd.to_numeric(df_claim[col], errors='raise', downcast=None)

In [105]:
df_claim[['AYear','AMonth','AHour','Amin']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91311 entries, 0 to 91310
Data columns (total 4 columns):
AYear     91311 non-null int64
AMonth    91311 non-null int64
AHour     91311 non-null int64
Amin      91311 non-null int64
dtypes: int64(4)
memory usage: 6.0 MB


In [106]:
df_claim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91311 entries, 0 to 91310
Columns: 178 entries, Policy_Number to DeductibleC_mean
dtypes: datetime64[ns](3), float64(21), int64(70), object(1), uint8(83)
memory usage: 76.6+ MB


### backup data

In [107]:
# df_claim.to_csv('../Tbrain_Insurance/df_claim.csv', index = False)

### categorize columns

## Combine

#### define combine with first, mean, sum, count, std for each columns 

In [123]:
df_claim[df_claim.columns[115:175]].tail()

,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber,AF_A,AF_B,AF_C,AF_A_sum,AF_A_sum,AF_B_sum,AF_B_sum,AF_C_sum,AF_C_sum,AF_A_mean,AF_B_mean,AF_C_mean,DeductibleA,DeductibleB,DeductibleC,AF_A_sum,AF_A_sum,AF_B_sum,AF_B_sum,AF_C_sum,AF_C_sum,Deductible_mean
91306,15762,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15762,0,0,0,0,0,15762,0,0,0,0,0,15762,0,0,0,0,0,2015,11,6,11,0,58.879452,1.0,1.0,100.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,100.0,0.000000,0.0,0,0,0,100.0,100.0,0.0,0.0,0.0,0.0,0.0
91307,0,0,900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,900,0,0,0,0,0,900,0,0,0,0,0,900,0,0,0,2016,12,3,14,0,50.120548,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91308,0,0,1050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1050,0,0,0,0,0,1050,0,0,0,0,0,1050,0,0,0,2016,9,3,14,0,32.526027,3.0,3.0,0.0,100.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,33.333333,0.0,0,0,0,0.0,0.0,100.0,100.0,0.0,0.0,0.0
91309,0,0,1050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1050,0,0,0,0,0,1050,0,0,0,0,0,1050,0,0,0,2016,9,3,14,0,32.526027,3.0,3.0,0.0,100.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,33.333333,0.0,0,0,0,0.0,0.0,100.0,100.0,0.0,0.0,0.0
91310,0,0,1050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1050,0,0,0,0,0,1050,0,0,0,0,0,1050,0,0,0,2016,9,3,14,0,32.526027,3.0,3.0,0.0,100.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,33.333333,0.0,0,0,0,0.0,0.0,100.0,100.0,0.0,0.0,0.0


In [211]:
df_claim[df_claim.columns[:-10]].head()

,Policy_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,Deductible,number_of_claimants,Accident_Time,Claim_Number_unuique,Claim_Number_count,Claim_Number_typecount,Claim_Number_typenunique,Nature_of_the_claim_1,Nature_of_the_claim_2,group_A,group_B,group_C,Claim_Number_type1count,Claim_Number_type2count,Claim_Number_typeAcount,Claim_Number_typeBcount,Claim_Number_typeCcount,Claim_Number_type1Acount,Claim_Number_type1Bcount,Claim_Number_type1Ccount,Claim_Number_type2Acount,Claim_Number_type2Bcount,Claim_Number_type2Ccount,Claim_Number_type1UniCount,Claim_Number_type2UniCount,Claim_Number_typeAUniCount,Claim_Number_typeBUniCount,Claim_Number_typeCUniCount,Claim_Number_type1UniCcount,Claim_Number_type2UniCcount,Driver's_Gender_1,Driver's_Gender_2,Driver's_Relationship_with_Insured_1,Driver's_Relationship_with_Insured_2,Driver's_Relationship_with_Insured_3,Driver's_Relationship_with_Insured_4,Driver's_Relationship_with_Insured_5,Driver's_Relationship_with_Insured_6,Driver's_Relationship_with_Insured_7,Marital_Status_of_Driver_1,Marital_Status_of_Driver_2,Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c,Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17,Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319,Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b,Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8,Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef,Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a,Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554,Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0,Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b,Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4,Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63,Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2,Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113,Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812,"Claim_Status_(close,_open,_reopen_etc)_0","Claim_Status_(close,_open,_reopen_etc)_1",Coverage_00I,Coverage_01A,Coverage_02K,Coverage_03L,Coverage_04M,Coverage_05E,Coverage_05N,Coverage_06F,Coverage_07P,Coverage_08H,Coverage_09@,Coverage_10A,Coverage_12L,Coverage_14E,Coverage_14N,Coverage_15F,Coverage_15O,Coverage_16G,Coverage_16P,Coverage_18@,Coverage_18I,Coverage_20B,Coverage_20K,Coverage_25G,Coverage_26H,Coverage_29B,Coverage_29K,Coverage_32N,Coverage_33F,Coverage_33O,Coverage_34P,Coverage_35H,Coverage_36I,Coverage_40M,Coverage_41E,Coverage_41N,Coverage_42F,Coverage_45@,Coverage_46A,Coverage_51O,Coverage_55J,Coverage_56B,Coverage_56K,Coverage_57C,Coverage_57L,Coverage_66L,Coverage_70G,Coverage_70P,PL_1A,PL_2A,PL_1B,PL_2B,PL_1C,PL_2C,PE_1A,PE_2A,PE_1B,PE_2B,PE_1C,PE_2C,Sub_1A,Sub_2A,Sub_1B,Sub_2B,Sub_1C,Sub_2C,1A_Sum_1,2A_Sum_1,1B_Sum_1,2B_Sum_1,1C_Sum_1,2C_Sum_1,1A_Sum_2,2A_Sum_2,1B_Sum_2,2B_Sum_2,1C_Sum_2,2C_Sum_2,1A_Sum_3,2A_Sum_3,1B_Sum_3,2B_Sum_3,1C_Sum_3,2C_Sum_3,AYear,AMonth,Acc_DayOfWeek,AHour,Amin,Dr_age,Claim_Number_onPlateNumber,Policy_Number_onPlateNumber,AF_A,AF_B,AF_C,AF_A_sum,AF_A_sum,AF_B_sum,AF_B_sum,AF_C_sum,AF_C_sum,AF_A_mean,AF_B_mean,AF_C_mean
0,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,0,0,56705,0,0,0,2015,1,3,8,0,47.619178,5.0,5.0,0.0,100.0,0.0,200.0,200.0,100.0,100.0,0.0,0.0,40.0,20.0,0.0
1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1969-06-01,2015-01-01,56705,0,0,100.0,3000,1.981,1900-01-01 08:00:00,2,5,5,2,1,0,0,1,0,5,0,0,5,0,0,5,0,0,0,0,2,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0

In [335]:
ref_col = ['Policy_Number']

first_cols = ['Policy_Number',
              'DOB_of_Driver','Accident_Date','Accident_Time','Claim_Number_unuique', 'Claim_Number_count',
              'Claim_Number_typecount', 'Claim_Number_typenunique', 'Nature_of_the_claim_1', 
              'Nature_of_the_claim_2', 'group_A', 'group_B', 'group_C', 'Claim_Number_type1count', 
              'Claim_Number_type2count', 'Claim_Number_typeAcount', 'Claim_Number_typeBcount',
              'Claim_Number_typeCcount', 'Claim_Number_type1Acount', 'Claim_Number_type1Bcount',
              'Claim_Number_type1Ccount', 'Claim_Number_type2Acount', 'Claim_Number_type2Bcount', 
              'Claim_Number_type2Ccount', 'Claim_Number_type1UniCount', 'Claim_Number_type2UniCount',
              'Claim_Number_typeAUniCount', 'Claim_Number_typeBUniCount', 'Claim_Number_typeCUniCount',
              'Claim_Number_type1UniCcount', 'Claim_Number_type2UniCcount', "Driver's_Gender_1", 
              "Driver's_Gender_2", "Driver's_Relationship_with_Insured_1", "Driver's_Relationship_with_Insured_2", 
              "Driver's_Relationship_with_Insured_3", "Driver's_Relationship_with_Insured_4", 
              "Driver's_Relationship_with_Insured_5", "Driver's_Relationship_with_Insured_6",
              "Driver's_Relationship_with_Insured_7", 'Marital_Status_of_Driver_1', 'Marital_Status_of_Driver_2',
              'Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c', 
              'Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17', 
              'Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319', 
              'Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b',
              'Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673', 
              'Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8', 
              'Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef', 
              'Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a', 
              'Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554', 
              'Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0', 
              'Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b', 
              'Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c', 
              'Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4', 
              'Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63', 
              'Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2', 
              'Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113', 
              'Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812', 
              'PL_1A', 'PL_2A','PL_1B', 'PL_2B', 'PL_1C','PL_2C', 'PE_1A', 
              'PE_2A', 'PE_1B', 'PE_2B', 'PE_1C', 'PE_2C', 'Sub_1A', 'Sub_2A',
              'Sub_1B', 'Sub_2B', 'Sub_1C', 'Sub_2C', '1A_Sum_1', '2A_Sum_1', 
              '1B_Sum_1', '2B_Sum_1', '1C_Sum_1', '2C_Sum_1', '1A_Sum_2', '2A_Sum_2', 
              '1B_Sum_2', '2B_Sum_2', '1C_Sum_2', '2C_Sum_2', '1A_Sum_3', '2A_Sum_3', 
              '1B_Sum_3', '2B_Sum_3', '1C_Sum_3', '2C_Sum_3','AYear','AMonth', 'Acc_DayOfWeek',
              'AHour', 'Amin', 'Dr_age', 'Claim_Number_onPlateNumber', 'Policy_Number_onPlateNumber', 
              'AF_A', 'AF_B', 'AF_C', 'AF_A_sum', 'AF_B_sum', 'AF_C_sum', 'AF_A_mean', 'AF_B_mean', 
              'AF_C_mean', 'DeductibleA', 'DeductibleB', 'DeductibleC', 'Deductible_mean','DeductibleA_mean',
              'DeductibleB_mean', 'DeductibleC_mean']



sum_cols = ['Policy_Number',
            'Paid_Loss_Amount','paid_Expenses_Amount', 'Salvage_or_Subrogation?','At_Fault?','Deductible',
            'number_of_claimants','Cause_of_Loss_0716d9708d321ffb6a00818614779e779925365c',
       'Cause_of_Loss_12c6fc06c99a462375eeb3f43dfd832b08ca9e17',
       'Cause_of_Loss_1574bddb75c78a6fd2251d61e2993b5146201319',
       'Cause_of_Loss_17ba0791499db908433b80f37c5fbc89b870084b',
       'Cause_of_Loss_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673',
       'Cause_of_Loss_472b07b9fcf2c2451e8781e944bf5f77cd8457c8',
       'Cause_of_Loss_4d134bc072212ace2df385dae143139da74ec0ef',
       'Cause_of_Loss_76546f9a641ede2beab506b96df1688d889e629a',
       'Cause_of_Loss_7b52009b64fd0a2a49e6d8a939753077792b0554',
       'Cause_of_Loss_9a79be611e0267e1d943da0737c6c51be67865a0',
       'Cause_of_Loss_af3e133428b9e25c55bc59fe534248e6a0c0f17b',
       'Cause_of_Loss_b6589fc6ab0dc82cf12099d1c2d40ab994e8410c',
    'Cause_of_Loss_bd307a3ec329e10a2cff8fb87480823da114f8f4',
       'Cause_of_Loss_c425c670244a64c875ef504945726c6ce2f74b63',
       'Cause_of_Loss_d435a6cdd786300dff204ee7c2ef942d3e9034e2',
       'Cause_of_Loss_e1822db470e60d090affd0956d743cb0e7cdf113',
       'Cause_of_Loss_f1abd670358e036c31296e66b3b66c382ac00812',
            'Claim_Status_(close,_open,_reopen_etc)_0',
       'Claim_Status_(close,_open,_reopen_etc)_1','Coverage_00I',
       'Coverage_01A', 'Coverage_02K', 'Coverage_03L', 'Coverage_04M',
       'Coverage_05E', 'Coverage_05N', 'Coverage_06F', 'Coverage_07P',
       'Coverage_08H', 'Coverage_09@', 'Coverage_10A', 'Coverage_12L',
       'Coverage_14E', 'Coverage_14N', 'Coverage_15F', 'Coverage_15O',
       'Coverage_16G', 'Coverage_16P', 'Coverage_18@', 'Coverage_18I',
       'Coverage_20B', 'Coverage_20K', 'Coverage_25G', 'Coverage_26H',
       'Coverage_29B', 'Coverage_29K', 'Coverage_32N', 'Coverage_33F',
       'Coverage_33O', 'Coverage_34P', 'Coverage_35H', 'Coverage_36I',
       'Coverage_40M', 'Coverage_41E', 'Coverage_41N', 'Coverage_42F',
       'Coverage_45@', 'Coverage_46A', 'Coverage_51O', 'Coverage_55J',
       'Coverage_56B', 'Coverage_56K', 'Coverage_57C', 'Coverage_57L',
       'Coverage_66L', 'Coverage_70G', 'Coverage_70P','PL_1A', 'PL_2A',
       'PL_1B', 'PL_2B', 'PL_1C','PL_2C', 'PE_1A', 'PE_2A', 'PE_1B', 'PE_2B', 'PE_1C', 'PE_2C', 'Sub_1A',
       'Sub_2A', 'Sub_1B', 'Sub_2B', 'Sub_1C', 'Sub_2C', '1A_Sum_1',
       '2A_Sum_1', '1B_Sum_1', '2B_Sum_1', '1C_Sum_1', '2C_Sum_1', '1A_Sum_2',
       '2A_Sum_2', '1B_Sum_2', '2B_Sum_2', '1C_Sum_2', '2C_Sum_2', '1A_Sum_3',
       '2A_Sum_3', '1B_Sum_3', '2B_Sum_3', '1C_Sum_3', '2C_Sum_3', 'AYear',
       'AMonth', 'Acc_DayOfWeek', 'AHour', 'Amin', 'Dr_age',
       'Claim_Number_onPlateNumber', 'Policy_Number_onPlateNumber', 'AF_A',
       'AF_B', 'AF_C', 'AF_A_sum', 'AF_B_sum', 'AF_C_sum', 'AF_A_mean',
       'AF_B_mean', 'AF_C_mean', 'DeductibleA', 'DeductibleB', 'DeductibleC',
        'Deductible_mean',
       'DeductibleA_mean', 'DeductibleB_mean', 'DeductibleC_mean']
    
    



mean_cols = ['Policy_Number',
             'Paid_Loss_Amount','paid_Expenses_Amount', "Salvage_or_Subrogation?",'At_Fault?',
             'Deductible', 'number_of_claimants',"Claim_Status_(close,_open,_reopen_etc)_0",
             'Claim_Status_(close,_open,_reopen_etc)_1','Coverage_00I','Coverage_01A', 'Coverage_02K', 
             'Coverage_03L', 'Coverage_04M','Coverage_05E', 'Coverage_05N', 'Coverage_06F', 'Coverage_07P',
             'Coverage_08H', 'Coverage_09@', 'Coverage_10A', 'Coverage_12L','Coverage_14E', 'Coverage_14N',
             'Coverage_15F', 'Coverage_15O','Coverage_16G', 'Coverage_16P', 'Coverage_18@', 'Coverage_18I',
             'Coverage_20B', 'Coverage_20K', 'Coverage_25G', 'Coverage_26H','Coverage_29B', 'Coverage_29K', 
             'Coverage_32N', 'Coverage_33F','Coverage_33O', 'Coverage_34P', 'Coverage_35H', 'Coverage_36I',
             'Coverage_40M', 'Coverage_41E', 'Coverage_41N', 'Coverage_42F','Coverage_45@', 'Coverage_46A', 
             'Coverage_51O', 'Coverage_55J','Coverage_56B', 'Coverage_56K', 'Coverage_57C', 'Coverage_57L',
             'Coverage_66L', 'Coverage_70G', 'Coverage_70P','PL_1A', 'PL_2A','PL_1B', 'PL_2B', 'PL_1C','PL_2C',
             'PE_1A', 'PE_2A', 'PE_1B', 'PE_2B', 'PE_1C', 'PE_2C', 'Sub_1A','Sub_2A', 'Sub_1B', 'Sub_2B', 
             'Sub_1C', 'Sub_2C', '1A_Sum_1','2A_Sum_1', '1B_Sum_1', '2B_Sum_1', '1C_Sum_1', '2C_Sum_1', 
             '1A_Sum_2','2A_Sum_2', '1B_Sum_2', '2B_Sum_2', '1C_Sum_2', '2C_Sum_2', '1A_Sum_3','2A_Sum_3',
             '1B_Sum_3', '2B_Sum_3', '1C_Sum_3', '2C_Sum_3', 'AYear','AMonth', 'Acc_DayOfWeek', 'AHour', 
             'Amin', 'Dr_age', 'Claim_Number_onPlateNumber', 'Policy_Number_onPlateNumber','AF_A', 'AF_B',
             'AF_C', 'AF_A_sum', 'AF_B_sum', 'AF_C_sum', 'AF_A_mean','AF_B_mean', 'AF_C_mean', 'DeductibleA',
             'DeductibleB', 'DeductibleC', 'Deductible_mean','DeductibleA_mean', 'DeductibleB_mean', 
             'DeductibleC_mean']
    
    
    

count_cols = [''] 

std_cols = ['']
       

In [336]:
print(len(first_cols))
print(len(set(first_cols)))
print(len(sum_cols))
print(len(set(sum_cols)))
print(len(mean_cols))
print(len(set(mean_cols)))

119
119
134
134
117
117


## combine (first sum count mean std)

In [293]:
df_policy_unique = pd.read_csv('../Tbrain_Insurance/df_policy_unuique.csv')
print(df_policy_unique.shape)
df_policy_unique.head()

(351273, 222)


,Policy_Number,Next_Premium,Cancellation,Engine_Displacement_(Cubic_Centimeter),qpt,Replacement_cost_of_insured_vehicle,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,ibirth,fequipment1,fequipment2,fequipment3,fequipment5,fequipment9,nequipment9,Policy_Number_Count,Policy_Number_Num,Vehicle_identifier_Num,lia_acc,V_age,cont_ratio,Premium_GroupSum,Premium_SumA,Premium_SumB,Premium_SumC,Premium_SumO,dbirth_new,equip_sum,Num_GroupC_onPN,Num_GroupB_onPN,Num_GroupA_onPN,Num_GroupC_onIID,Num_GroupB_onIID,Num_GroupA_onIID,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,fassured_1,fassured_2,fassured_3,fassured_6,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fsex_,fsex_1,fsex_2,fmarriage_,fmarriage_1,fmarriage_2,i_age,d_age,Insured_Amount1_ASum,Insured_Amount1_BSum,Insured_Amount1_CSum,Insured_Amount2_ASum,Insured_Amount2_BSum,Insured_Amount2_CSum,Insured_Amount3_ASum,Insured_Amount3_BSum,Insured_Amount3_CSum,Insured_AmountO_ASum,Insured_AmountO_BSum,Insured_AmountO_CSum,Insured_Amount1_AStd,Insured_Amount1_BStd,Insured_Amount1_CStd,Insured_Amount2_AStd,Insured_Amount2_BStd,Insured_Amount2_CStd,Insured_Amount3_AStd,Insured_Amount3_BStd,Insured_Amount3_CStd,Insured_AmountO_AStd,Insured_AmountO_BStd,Insured_AmountO_CStd,Insured_Amount1_AMean,Insured_Amount1_BMean,Insured_Amount1_CMean,Insured_Amount2_AMean,Insured_Amount2_BMean,Insured_Amount2_CMean,Insured_Amount3_AMean,Insured_Amount3_BMean,Insured_Amount3_CMean,Insured_AmountO_AMean,Insured_AmountO_BMean,Insured_AmountO_CMean,Premium,Cancelled,CD_absolute_GroupMean,CD_pct_GroupMean,CD_absolute2_GroupMean,CD_pct2_GroupMean,CD_absolute_GroupSum,CD_pct_GroupSum,CD_absolute2_GroupSum,CD_pct2_GroupSum,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_C

In [307]:
df_all = df_policy_unique

In [308]:
df = df_claim
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91311 entries, 0 to 91310
Columns: 178 entries, Policy_Number to DeductibleC_mean
dtypes: datetime64[ns](3), float64(21), int64(70), object(1), uint8(83)
memory usage: 76.6+ MB


In [337]:
# df -> df_grouped -> combine with df_all

# first
df_grouped = df.groupby(ref_col).first().reset_index()[first_cols] # #

print('unique:',df[ref_col].nunique()) 
print('grouped: ', df_grouped.shape)  # == 35,895 unique Policy Number
 

## all = all+ grouped
print('before combine:',df_all.shape)
df_all= df_all.merge(df_grouped , how='left', on ='Policy_Number', suffixes=('', '_first'))#
print('after combine:', df_all.shape)
df_all.head() 

unique: Policy_Number    35895
dtype: int64
grouped:  (35895, 122)
before combine: (351273, 222)
after combine: (351273, 343)


,Policy_Number,Next_Premium,Cancellation,Engine_Displacement_(Cubic_Centimeter),qpt,Replacement_cost_of_insured_vehicle,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,ibirth,fequipment1,fequipment2,fequipment3,fequipment5,fequipment9,nequipment9,Policy_Number_Count,Policy_Number_Num,Vehicle_identifier_Num,lia_acc,V_age,cont_ratio,Premium_GroupSum,Premium_SumA,Premium_SumB,Premium_SumC,Premium_SumO,dbirth_new,equip_sum,Num_GroupC_onPN,Num_GroupB_onPN,Num_GroupA_onPN,Num_GroupC_onIID,Num_GroupB_onIID,Num_GroupA_onIID,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,fassured_1,fassured_2,fassured_3,fassured_6,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fsex_,fsex_1,fsex_2,fmarriage_,fmarriage_1,fmarriage_2,i_age,d_age,Insured_Amount1_ASum,Insured_Amount1_BSum,Insured_Amount1_CSum,Insured_Amount2_ASum,Insured_Amount2_BSum,Insured_Amount2_CSum,Insured_Amount3_ASum,Insured_Amount3_BSum,Insured_Amount3_CSum,Insured_AmountO_ASum,Insured_AmountO_BSum,Insured_AmountO_CSum,Insured_Amount1_AStd,Insured_Amount1_BStd,Insured_Amount1_CStd,Insured_Amount2_AStd,Insured_Amount2_BStd,Insured_Amount2_CStd,Insured_Amount3_AStd,Insured_Amount3_BStd,Insured_Amount3_CStd,Insured_AmountO_AStd,Insured_AmountO_BStd,Insured_AmountO_CStd,Insured_Amount1_AMean,Insured_Amount1_BMean,Insured_Amount1_CMean,Insured_Amount2_AMean,Insured_Amount2_BMean,Insured_Amount2_CMean,Insured_Amount3_AMean,Insured_Amount3_BMean,Insured_Amount3_CMean,Insured_AmountO_AMean,Insured_AmountO_BMean,Insured_AmountO_CMean,Premium,Cancelled,CD_absolute_GroupMean,CD_pct_GroupMean,CD_absolute2_GroupMean,CD_pct2_GroupMean,CD_absolute_GroupSum,CD_pct_GroupSum,CD_absolute2_GroupSum,CD_pct2_GroupSum,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_C

In [338]:
# df -> df_grouped -> combine with df_all

# mean
df_grouped = df.groupby(ref_col).mean().reset_index()[mean_cols]  # #
print('unique:',df[ref_col].nunique()) 
print('grouped: ', df_grouped.shape)  # == 35,895 unique Policy Number
 
## all = all+ grouped
print('before combine:',df_all.shape)
df_all= df_all.merge(df_grouped , how='left', on ='Policy_Number', suffixes=('', '_mean'))#
print('after combine:', df_all.shape)
df_all.head()

unique: Policy_Number    35895
dtype: int64
grouped:  (35895, 120)
before combine: (351273, 343)
after combine: (351273, 462)


,Policy_Number,Next_Premium,Cancellation,Engine_Displacement_(Cubic_Centimeter),qpt,Replacement_cost_of_insured_vehicle,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,ibirth,fequipment1,fequipment2,fequipment3,fequipment5,fequipment9,nequipment9,Policy_Number_Count,Policy_Number_Num,Vehicle_identifier_Num,lia_acc,V_age,cont_ratio,Premium_GroupSum,Premium_SumA,Premium_SumB,Premium_SumC,Premium_SumO,dbirth_new,equip_sum,Num_GroupC_onPN,Num_GroupB_onPN,Num_GroupA_onPN,Num_GroupC_onIID,Num_GroupB_onIID,Num_GroupA_onIID,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,fassured_1,fassured_2,fassured_3,fassured_6,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fsex_,fsex_1,fsex_2,fmarriage_,fmarriage_1,fmarriage_2,i_age,d_age,Insured_Amount1_ASum,Insured_Amount1_BSum,Insured_Amount1_CSum,Insured_Amount2_ASum,Insured_Amount2_BSum,Insured_Amount2_CSum,Insured_Amount3_ASum,Insured_Amount3_BSum,Insured_Amount3_CSum,Insured_AmountO_ASum,Insured_AmountO_BSum,Insured_AmountO_CSum,Insured_Amount1_AStd,Insured_Amount1_BStd,Insured_Amount1_CStd,Insured_Amount2_AStd,Insured_Amount2_BStd,Insured_Amount2_CStd,Insured_Amount3_AStd,Insured_Amount3_BStd,Insured_Amount3_CStd,Insured_AmountO_AStd,Insured_AmountO_BStd,Insured_AmountO_CStd,Insured_Amount1_AMean,Insured_Amount1_BMean,Insured_Amount1_CMean,Insured_Amount2_AMean,Insured_Amount2_BMean,Insured_Amount2_CMean,Insured_Amount3_AMean,Insured_Amount3_BMean,Insured_Amount3_CMean,Insured_AmountO_AMean,Insured_AmountO_BMean,Insured_AmountO_CMean,Premium,Cancelled,CD_absolute_GroupMean,CD_pct_GroupMean,CD_absolute2_GroupMean,CD_pct2_GroupMean,CD_absolute_GroupSum,CD_pct_GroupSum,CD_absolute2_GroupSum,CD_pct2_GroupSum,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_C

In [339]:
# df -> df_grouped -> combine with df_all

# sum
df_grouped = df.groupby(ref_col).sum().reset_index()[sum_cols]  # #
print('unique:',df[ref_col].nunique()) 
print('grouped: ', df_grouped.shape)  # == 35,895 unique Policy Number
 
## all = all+ grouped
print('before combine:',df_all.shape)
df_all= df_all.merge(df_grouped , how='left', on ='Policy_Number', suffixes=('', '_sum'))#
print('after combine:', df_all.shape)
df_all.head() 

unique: Policy_Number    35895
dtype: int64
grouped:  (35895, 137)
before combine: (351273, 462)
after combine: (351273, 598)


,Policy_Number,Next_Premium,Cancellation,Engine_Displacement_(Cubic_Centimeter),qpt,Replacement_cost_of_insured_vehicle,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,ibirth,fequipment1,fequipment2,fequipment3,fequipment5,fequipment9,nequipment9,Policy_Number_Count,Policy_Number_Num,Vehicle_identifier_Num,lia_acc,V_age,cont_ratio,Premium_GroupSum,Premium_SumA,Premium_SumB,Premium_SumC,Premium_SumO,dbirth_new,equip_sum,Num_GroupC_onPN,Num_GroupB_onPN,Num_GroupA_onPN,Num_GroupC_onIID,Num_GroupB_onIID,Num_GroupA_onIID,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,fassured_1,fassured_2,fassured_3,fassured_6,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fsex_,fsex_1,fsex_2,fmarriage_,fmarriage_1,fmarriage_2,i_age,d_age,Insured_Amount1_ASum,Insured_Amount1_BSum,Insured_Amount1_CSum,Insured_Amount2_ASum,Insured_Amount2_BSum,Insured_Amount2_CSum,Insured_Amount3_ASum,Insured_Amount3_BSum,Insured_Amount3_CSum,Insured_AmountO_ASum,Insured_AmountO_BSum,Insured_AmountO_CSum,Insured_Amount1_AStd,Insured_Amount1_BStd,Insured_Amount1_CStd,Insured_Amount2_AStd,Insured_Amount2_BStd,Insured_Amount2_CStd,Insured_Amount3_AStd,Insured_Amount3_BStd,Insured_Amount3_CStd,Insured_AmountO_AStd,Insured_AmountO_BStd,Insured_AmountO_CStd,Insured_Amount1_AMean,Insured_Amount1_BMean,Insured_Amount1_CMean,Insured_Amount2_AMean,Insured_Amount2_BMean,Insured_Amount2_CMean,Insured_Amount3_AMean,Insured_Amount3_BMean,Insured_Amount3_CMean,Insured_AmountO_AMean,Insured_AmountO_BMean,Insured_AmountO_CMean,Premium,Cancelled,CD_absolute_GroupMean,CD_pct_GroupMean,CD_absolute2_GroupMean,CD_pct2_GroupMean,CD_absolute_GroupSum,CD_pct_GroupSum,CD_absolute2_GroupSum,CD_pct2_GroupSum,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_C

In [347]:
print(df_all.columns[:60])
print(df_all.columns[60:120])
print(df_all.columns[120:180])
print(df_all.columns[180:240])
print(df_all.columns[240:300])
print(df_all.columns[300:360])
print(df_all.columns[360:420])
print(df_all.columns[420:480])
print(df_all.columns[480:540])
print(df_all.columns[540:600])

Index(['Policy_Number', 'Next_Premium', 'Cancellation',
       'Engine_Displacement_(Cubic_Centimeter)', 'qpt',
       'Replacement_cost_of_insured_vehicle',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'ibirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment5', 'fequipment9', 'nequipment9', 'Policy_Number_Count',
       'Policy_Number_Num', 'Vehicle_identifier_Num', 'lia_acc', 'V_age',
       'cont_ratio', 'Premium_GroupSum', 'Premium_SumA', 'Premium_SumB',
       'Premium_SumC', 'Premium_SumO', 'dbirth_new', 'equip_sum',
       'Num_GroupC_onPN', 'Num_GroupB_onPN', 'Num_GroupA_onPN',
       'Num_GroupC_onIID', 'Num_GroupB_onIID', 'Num_GroupA_onIID',
       'Imported_or_Domestic_Car_10', 'Imported_or_Domestic_Car_20',
       'Imported_or_Domestic_Car_21', 'Imported_or_Domestic_Car_22',
       'Imported_or_Domestic_Car_23', 'Imported_or_Domestic_Car_24',
       'Imported_or_Domestic_Car_30', 'Imported_or_Domestic_Car_

In [ ]:
#    / InsuredGroup InsuredO PremiumGroup PremiumO   **********
'''
       '1A_Sum_1_sum','2A_Sum_1_sum', '1B_Sum_1_sum',  (sum/mean)
       '2B_Sum_1_sum', '1C_Sum_1_sum','2C_Sum_1_sum', 
       '1A_Sum_2_sum', '2A_Sum_2_sum', '1B_Sum_2_sum',
       '2B_Sum_2_sum', '1C_Sum_2_sum', '2C_Sum_2_sum', 
       '1A_Sum_3_sum','2A_Sum_3_sum', '1B_Sum_3_sum', 
       '2B_Sum_3_sum', '1C_Sum_3_sum', '2C_Sum_3_sum'
       
       'Deductible', 'DeductibleA','DeductibleB','DeductibleC'
       
       -------------------------------------------------------------------------------------------
       'Premium_GroupSum', 'Premium_SumA', 'Premium_SumB',
       'Premium_SumC', 'Premium_SumO'
       
       'Insured_Amount1_ASum', 'Insured_Amount1_BSum',
       'Insured_Amount1_CSum', 'Insured_Amount2_ASum', 'Insured_Amount2_BSum',
       'Insured_Amount2_CSum', 'Insured_Amount3_ASum', 'Insured_Amount3_BSum',
       'Insured_Amount3_CSum', 'Insured_AmountO_ASum', 'Insured_AmountO_BSum',
       'Insured_AmountO_CSum', 
       
       
'''

In [352]:
df_all['1ASD1Sum'] = df_all['1A_Sum_1_sum'] - df_all['DeductibleA']
df_all['2ASD1Sum'] = df_all['2A_Sum_1_sum'] - df_all['DeductibleA']
df_all['1BSD1Sum'] = df_all['1B_Sum_1_sum'] - df_all['DeductibleB']
df_all['2BSD1Sum'] = df_all['2B_Sum_1_sum'] - df_all['DeductibleB']
df_all['1CSD1Sum'] = df_all['1C_Sum_1_sum'] - df_all['DeductibleC']
df_all['2CSD1Sum'] = df_all['2C_Sum_1_sum'] - df_all['DeductibleC']

df_all['1ASD2Sum'] = df_all['1A_Sum_2_sum'] - df_all['DeductibleA']
df_all['2ASD2Sum'] = df_all['2A_Sum_2_sum'] - df_all['DeductibleA']
df_all['1BSD2Sum'] = df_all['1B_Sum_2_sum'] - df_all['DeductibleB']
df_all['2BSD2Sum'] = df_all['2B_Sum_2_sum'] - df_all['DeductibleB']
df_all['1CSD2Sum'] = df_all['1C_Sum_2_sum'] - df_all['DeductibleC']
df_all['2CSD2Sum'] = df_all['2C_Sum_2_sum'] - df_all['DeductibleC']

df_all['1ASD3Sum'] = df_all['1A_Sum_3_sum'] - df_all['DeductibleA']
df_all['2ASD3Sum'] = df_all['2A_Sum_3_sum'] - df_all['DeductibleA']
df_all['1BSD3Sum'] = df_all['1B_Sum_3_sum'] - df_all['DeductibleB']
df_all['2BSD3Sum'] = df_all['2B_Sum_3_sum'] - df_all['DeductibleB']
df_all['1CSD3Sum'] = df_all['1C_Sum_3_sum'] - df_all['DeductibleC']
df_all['2CSD3Sum'] = df_all['2C_Sum_3_sum'] - df_all['DeductibleC']

df_all['1ASD1mean'] = df_all['1A_Sum_1_mean'] - df_all['DeductibleA']
df_all['2ASD1mean'] = df_all['2A_Sum_1_mean'] - df_all['DeductibleA']
df_all['1BSD1mean'] = df_all['1B_Sum_1_mean'] - df_all['DeductibleB']
df_all['2BSD1mean'] = df_all['2B_Sum_1_mean'] - df_all['DeductibleB']
df_all['1CSD1mean'] = df_all['1C_Sum_1_mean'] - df_all['DeductibleC']
df_all['2CSD1mean'] = df_all['2C_Sum_1_mean'] - df_all['DeductibleC']

df_all['1ASD2mean'] = df_all['1A_Sum_2_mean'] - df_all['DeductibleA']
df_all['2ASD2mean'] = df_all['2A_Sum_2_mean'] - df_all['DeductibleA']
df_all['1BSD2mean'] = df_all['1B_Sum_2_mean'] - df_all['DeductibleB']
df_all['2BSD2mean'] = df_all['2B_Sum_2_mean'] - df_all['DeductibleB']
df_all['1CSD2mean'] = df_all['1C_Sum_2_mean'] - df_all['DeductibleC']
df_all['2CSD2mean'] = df_all['2C_Sum_2_mean'] - df_all['DeductibleC']

df_all['1ASD3mean'] = df_all['1A_Sum_3_mean'] - df_all['DeductibleA']
df_all['2ASD3mean'] = df_all['2A_Sum_3_mean'] - df_all['DeductibleA']
df_all['1BSD3mean'] = df_all['1B_Sum_3_mean'] - df_all['DeductibleB']
df_all['2BSD3mean'] = df_all['2B_Sum_3_mean'] - df_all['DeductibleB']
df_all['1CSD3mean'] = df_all['1C_Sum_3_mean'] - df_all['DeductibleC']
df_all['2CSD3mean'] = df_all['2C_Sum_3_mean'] - df_all['DeductibleC']


In [353]:
df_all['1ASD1SumPO'] = df_all['1ASD1Sum']  / df_all['Premium_SumA']
df_all['2ASD1SumPO'] = df_all['2ASD1Sum']  / df_all['Premium_SumA']
df_all['1BSD1SumPO'] = df_all['1BSD1Sum']  / df_all['Premium_SumB']
df_all['2BSD1SumPO'] = df_all['2BSD1Sum'] / df_all['Premium_SumB']
df_all['1CSD1SumPO'] = df_all['1CSD1Sum'] / df_all['Premium_SumC']
df_all['2CSD1SumPO'] = df_all['2CSD1Sum']  / df_all['Premium_SumC']

df_all['1ASD2SumPO'] = df_all['1ASD2Sum']  / df_all['Premium_SumA']
df_all['2ASD2SumPO'] = df_all['2ASD2Sum']  / df_all['Premium_SumA']
df_all['1BSD2SumPO'] = df_all['1BSD2Sum']  / df_all['Premium_SumB']
df_all['2BSD2SumPO'] = df_all['2BSD2Sum']  / df_all['Premium_SumB']
df_all['1CSD2SumPO'] = df_all['1CSD2Sum']  / df_all['Premium_SumC']
df_all['2CSD2SumPO'] = df_all['2CSD2Sum']  / df_all['Premium_SumC']

df_all['1ASD3SumPO'] = df_all['1ASD3Sum'] / df_all['Premium_SumA']
df_all['2ASD3SumPO'] = df_all['2ASD3Sum']  / df_all['Premium_SumA']
df_all['1BSD3SumPO'] = df_all['1BSD3Sum']  / df_all['Premium_SumB']
df_all['2BSD3SumPO'] = df_all['2BSD3Sum']  / df_all['Premium_SumB']
df_all['1CSD3SumPO'] = df_all['1CSD3Sum']  / df_all['Premium_SumC']
df_all['2CSD3SumPO'] = df_all['2CSD3Sum']  / df_all['Premium_SumC']

df_all['1ASD1meanPO'] = df_all['1ASD1mean']  / df_all['Premium_SumA']
df_all['2ASD1meanPO'] = df_all['2ASD1mean']  / df_all['Premium_SumA']
df_all['1BSD1meanPO'] = df_all['1BSD1mean']  / df_all['Premium_SumB']
df_all['2BSD1meanPO'] = df_all['2BSD1mean']  / df_all['Premium_SumB']
df_all['1CSD1meanPO'] = df_all['1CSD1mean']  / df_all['Premium_SumC']
df_all['2CSD1meanPO'] = df_all['2CSD1mean']  / df_all['Premium_SumC']

df_all['1ASD2meanPO'] = df_all['1ASD2mean']  / df_all['Premium_SumA']
df_all['2ASD2meanPO'] = df_all['2ASD2mean']  / df_all['Premium_SumA']
df_all['1BSD2meanPO'] = df_all['1BSD2mean']  / df_all['Premium_SumB']
df_all['2BSD2meanPO'] = df_all['2BSD2mean']  / df_all['Premium_SumB']
df_all['1CSD2meanPO'] = df_all['1CSD2mean']  / df_all['Premium_SumC']
df_all['2CSD2meanPO'] = df_all['2CSD2mean']  / df_all['Premium_SumC']

df_all['1ASD3meanPO'] = df_all['1ASD3mean']  / df_all['Premium_SumA']
df_all['2ASD3meanPO'] = df_all['2ASD3mean']  / df_all['Premium_SumA']
df_all['1BSD3meanPO'] = df_all['1BSD3mean']  / df_all['Premium_SumB']
df_all['2BSD3meanPO'] = df_all['2BSD3mean']  / df_all['Premium_SumB']
df_all['1ASD3meanPO'] = df_all['1CSD3mean']  / df_all['Premium_SumC']
df_all['2CSD3meanPO'] = df_all['2CSD3mean']  / df_all['Premium_SumC']


In [355]:
df_all['1ASD1SumIO'] = df_all['1ASD1Sum']  / df_all['Insured_AmountO_ASum']
df_all['2ASD1SumIO'] = df_all['2ASD1Sum']  / df_all['Insured_AmountO_ASum']
df_all['1BSD1SumIO'] = df_all['1BSD1Sum']  / df_all['Insured_AmountO_BSum']
df_all['2BSD1SumIO'] = df_all['2BSD1Sum']  / df_all['Insured_AmountO_BSum']
df_all['1CSD1SumIO'] = df_all['1CSD1Sum']  / df_all['Insured_AmountO_CSum']
df_all['2CSD1SumIO'] = df_all['2CSD1Sum']  / df_all['Insured_AmountO_CSum']

df_all['1ASD2SumIO'] = df_all['1ASD2Sum']  / df_all['Insured_AmountO_ASum']
df_all['2ASD2SumIO'] = df_all['2ASD2Sum']  / df_all['Insured_AmountO_ASum']
df_all['1BSD2SumIO'] = df_all['1BSD2Sum']  / df_all['Insured_AmountO_BSum']
df_all['2BSD2SumIO'] = df_all['2BSD2Sum']  / df_all['Insured_AmountO_BSum']
df_all['1CSD2SumIO'] = df_all['1CSD2Sum']  / df_all['Insured_AmountO_CSum']
df_all['2CSD2SumIO'] = df_all['2CSD2Sum']  / df_all['Insured_AmountO_CSum']

df_all['1ASD3SumIO'] = df_all['1ASD3Sum']  / df_all['Insured_AmountO_ASum']
df_all['2ASD3SumIO'] = df_all['2ASD3Sum']  / df_all['Insured_AmountO_ASum']
df_all['1BSD3SumIO'] = df_all['1BSD3Sum']  / df_all['Insured_AmountO_BSum']
df_all['2BSD3SumIO'] = df_all['2BSD3Sum']  / df_all['Insured_AmountO_BSum']
df_all['1CSD3SumIO'] = df_all['1CSD3Sum']  / df_all['Insured_AmountO_CSum']
df_all['2CSD3SumIO'] = df_all['2CSD3Sum']  / df_all['Insured_AmountO_CSum']

df_all['1ASD1meanIO'] = df_all['1ASD1mean']  / df_all['Insured_AmountO_ASum']
df_all['2ASD1meanIO'] = df_all['2ASD1mean']  / df_all['Insured_AmountO_ASum']
df_all['1BSD1meanIO'] = df_all['1BSD1mean']  / df_all['Insured_AmountO_BSum']
df_all['2BSD1meanIO'] = df_all['2BSD1mean']  / df_all['Insured_AmountO_BSum']
df_all['1CSD1meanIO'] = df_all['1CSD1mean']  / df_all['Insured_AmountO_CSum']
df_all['2CSD1meanIO'] = df_all['2CSD1mean']  / df_all['Insured_AmountO_CSum']

df_all['1ASD2meanIO'] = df_all['1ASD2mean']  / df_all['Insured_AmountO_ASum']
df_all['2ASD2meanIO'] = df_all['2ASD2mean']  / df_all['Insured_AmountO_ASum']
df_all['1BSD2meanIO'] = df_all['1BSD2mean']  / df_all['Insured_AmountO_BSum']
df_all['2BSD2meanIO'] = df_all['2BSD2mean']  / df_all['Insured_AmountO_BSum']
df_all['1CSD2meanIO'] = df_all['1CSD2mean']  / df_all['Insured_AmountO_CSum']
df_all['2CSD2meanIO'] = df_all['2CSD2mean']  / df_all['Insured_AmountO_CSum']

df_all['1ASD3meanIO'] = df_all['1ASD3mean']  / df_all['Insured_AmountO_ASum']
df_all['2ASD3meanIO'] = df_all['2ASD3mean']  / df_all['Insured_AmountO_ASum']
df_all['1BSD3meanIO'] = df_all['1BSD3mean']  / df_all['Insured_AmountO_BSum']
df_all['2BSD3meanIO'] = df_all['2BSD3mean']  / df_all['Insured_AmountO_BSum']
df_all['1ASD3meanIO'] = df_all['1CSD3mean']  / df_all['Insured_AmountO_CSum']
df_all['2CSD3meanIO'] = df_all['2CSD3mean']  / df_all['Insured_AmountO_CSum']



In [356]:
df_all.to_csv('../Tbrain_Insurance/df_policy_claim_unuique.csv', index = False)

In [359]:
df_all.shape

(351273, 704)

In [358]:
df_all.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351273 entries, 0 to 351272
Data columns (total 704 columns):
Policy_Number                                                 object
Next_Premium                                                  int64
Cancellation                                                  int64
Engine_Displacement_(Cubic_Centimeter)                        int64
qpt                                                           float64
Replacement_cost_of_insured_vehicle                           float64
Multiple_Products_with_TmNewa_(Yes_or_No?)                    float64
lia_class                                                     int64
plia_acc                                                      float64
pdmg_acc                                                      float64
ibirth                                                        object
fequipment1                                                   int64
fequipment2                                                 

### seperate Nature and Juridical

In [78]:
df_policy_unique = pd.read_csv('../Tbrain_Insurance/df_policy_unuique.csv')
print(df_policy_unique.shape)
df_policy_unique.head()

(351273, 222)
